# 🧩 Lab 5 — Tracing & Resource-Aware Agents with OpenAI SDK

### 🎯 Goal
Learn how to **trace agent workflows** using OpenAI SDK and how to use **resources** as contextual knowledge for your Agentic AI system.

**Objectives**
- Understand what tracing is and why it matters.
- Learn to define and use resources inside an agent.
- Compare OpenAI native tracing vs LangSmith tracing.
- Observe agent → LLM → resource → tool interactions in action.

In [1]:
# 🪜 Step 1 — Setup and Configuration
from agents import Agent, Runner, function_tool
import os
from dotenv import load_dotenv

load_dotenv(override=True)

if not os.getenv('OPENAI_API_KEY'):
    print('⚠️ Please set your OPENAI_API_KEY environment variable before running.')
else:
    print('✅ API key loaded — ready to trace!')


✅ API key loaded — ready to trace!


## 🔍 What is Tracing?
Tracing helps you visualize how an agent executes a workflow step-by-step:
- Which LLMs were called and with what inputs
- Which tools were invoked and their outputs
- When resources were used for context

It provides a transparent view of the agent’s reasoning path and decision-making.

In [2]:
# 🧠 Step 2 — Define Tools
@function_tool
def send_email_tool(recipient: str, subject: str, body: str) -> str:
    """Simulates sending an email using SendGrid."""
    return f'📧 Email sent to {recipient} | Subject: {subject} | Body: {body[:60]}...'

@function_tool
def send_push_tool(message: str) -> str:
    """Simulates a push notification via Pushover."""
    return f'🔔 Push notification sent: {message[:80]}...'

@function_tool
def math_tool(expression: str) -> str:
    """Evaluates a simple math expression safely."""
    try:
        result = eval(expression, {"__builtins__": {}})
        return f'Result: {result}'
    except Exception as e:
        return f'Error evaluating expression: {e}'

@function_tool
def mini_llm_tool(query: str) -> str:
    """Simulates a small local LLM call (like Ollama)."""
    return f'🤖 Mini LLM response: Key insights about {query}.'

@function_tool
def read_project_scope() -> str:
    """Reads internal project scope document for agent context."""
    with open("resources/project_scope.txt", "r") as f:
        return f.read()

### 🧩 Tool Summary
| Tool | Purpose |
|------|----------|
| `send_email_tool` | Simulates email automation (SendGrid) |
| `send_push_tool` | Sends a quick push update (Pushover) |
| `math_tool` | Evaluates expressions |
| `mini_llm_tool` | Runs a lightweight LLM (Ollama-style) |

In [3]:
# 📘 Step 3 — Define a Resource
import os

os.makedirs('resources', exist_ok=True)

with open('resources/project_scope.txt', 'w') as f:
    f.write("""
Project Name: DPoint AI – Agentic AI Training & Workflow Automation Platform

Project Scope Overview:
DPoint AI is developing a modern Agentic AI training and automation platform designed for students, developers, and enterprise teams. The platform combines Large Language Models, tools, resources, and workflow orchestration to simplify building, testing, and deploying agent-based systems.

1. Core Objectives:
- Provide a hands-on learning environment to teach users how to build Agentic AI systems.
- Enable creation of multi-agent workflows that use tools, resources, and web actions.
- Offer real-time tracing and debugging using LangSmith-style observability.
- Demonstrate how LLMs interact with files, APIs, browser automation, and custom tools.

2. Key Features:
- Resource-aware agents that read files (e.g., project scope), retrieve context, and use it in decisions.
- Tool execution including browser actions, file operations, API calls, and custom plugins.
- Agent-to-agent handoff support for complex workflows.
- Built-in workflow tracing for education, demos, and enterprise visibility.
- Interactive exercises for:
  * LLM grounding
  * Prompt engineering
  * Multi-step reasoning
  * Tool calling
  * Debugging/tracing with LangSmith

3. Target Users:
- Students learning LLMs and Agentic AI
- Developers experimenting with agents, tools, and workflows
- Enterprise teams exploring automation and internal AI copilots
- Educators/trainers teaching modern LLM architectures

4. Expected Outcomes:
- Faster adoption of Agentic AI concepts in academic and corporate environments.
- A reusable training curriculum with labs, example projects, and guided workflows.
- A platform that demonstrates:
  * How LLM agents consume resources
  * How they summarize information
  * How they communicate updates (email, push notifications, etc.)

5. Business Impact:
- Standardizes how teams learn and prototype Agentic AI.
- Reduces onboarding time for AI developers.
- Enables enterprises to visualize and control LLM workflows with full traceability.
- Accelerates development of AI copilots and internal automation tools.
    """)

# resource = Resource(
#     name='Project Scope',
#     path='resources/project_scope.txt',
#     description='Context file with DPoint AI project scope and goals.'
# )

print('✅ Resource ready at resources/project_scope.txt')


✅ Resource ready at resources/project_scope.txt


### 📂 What are Resources?
Resources act as knowledge bases for Agents — text files, PDFs, or structured data they can consult while reasoning.

Here our resource is a project document used for contextual decision making.

In [4]:
# from agents import Resource

# resource = Resource(
#     name="Project Scope",
#     path="resources/project_scope.txt",
#     description="Context file with DPoint AI project scope and goals."
# )


In [22]:
from openai import OpenAI
client = OpenAI()

file = client.files.create(
    file=open("resources/project_scope.txt", "rb"),
    purpose="assistants" #  ['fine-tune', 'assistants', 'batch', 'user_data', 'vision', 'evals'] 
)


In [15]:
file

FileObject(id='file-U4JtniusYoSJuy3FaWGixG', bytes=2179, created_at=1763186588, filename='project_scope.txt', object='file', purpose='assistants', status='processed', expires_at=None, status_details=None)

In [ ]:
# ⚙️ Step 4 — Create the Agent
agent = Agent(
    name='TracingAgent',
    # option 1
    # instructions='Use the project resource for context before acting. Always explain reasoning before using tools.',
    # resources=[resource]

    # option 2
    # write instruction to use file name 
    # files=["resources/project_scope.txt"], 
    # files=['file-U4JtniusYoSJuy3FaWGixG'], #file.id

    # option 3
    # instruction = "use this for answering ur questions + {content of file}" - system prompt
    
    # option 4
    instructions="""Call read_project_scope() before doing anything. Use its content to understand the project and summarize it. 
    Always explain reasoning before using tools.""",

    

    model='gpt-4o-mini',
    tools=[send_email_tool, send_push_tool, math_tool, mini_llm_tool,read_project_scope]
)
print('✅ Agent created successfully!')

✅ Agent created successfully!


### 🧠 Agent Architecture
```
User → Agent → LLM → Resource → Tool 1 / Tool 2 / Tool 3 → Output
            ↳ Tracing Layer monitors each step
```
🧭 The **Tracing Layer** captures reasoning, tool calls and resource reads for debug visibility.

In [6]:
# 🚀 Step 5 — Run the Agent with Tracing
from agents import trace

prompt = 'Summarize the project scope and send an update to stakeholders via email and push notification.'

with trace(workflow_name='Tracing Demo — Resources in Action - with openai'):
    # result = Runner.run_sync(agent, prompt)
    result = await Runner.run(agent, prompt)
    print(result.final_output)

The updates have been successfully sent to the stakeholders:

- **Email**: An email detailing the DPoint AI project scope and objectives was sent to stakeholders.
- **Push Notification**: A brief update on the project development was also sent via push notification.

If you need any further assistance or additional updates, feel free to ask!


## 🔎 Step 6 — Tracing Types
### 🟢 OpenAI Native Tracing
- Enabled automatically with `with trace()` context.
- View traces at 👉 https://platform.openai.com/traces
```python
with trace(workflow_name='Agent Trace Demo'):
    result = Runner.run_sync(agent, user_prompt)
```

### 🔵 LangSmith Tracing (Advanced – Lab 6)
Add deep observability by enabling environment variables:
```bash
export LANGCHAIN_TRACING_V2='true'
export LANGCHAIN_ENDPOINT='https://api.smith.langchain.com'
export LANGCHAIN_API_KEY='your_key_here'
export LANGSMITH_PROJECT=''
```
```python
from langsmith import trace
with trace('LangSmith Demo'):
    result = Runner.run_sync(agent, prompt)
```

### 🧭 Tracing Flow Overview
1️⃣ User input → Agent reasoning
2️⃣ Agent references resources
3️⃣ LLM processes and plans
4️⃣ Tools execute actions (email, push, math)
5️⃣ Tracing logs all steps hierarchically

```
+-------------------------------------------------------------+
| TRACE VIEW                                                 |
|-------------------------------------------------------------|
| User Prompt → Agent Reasoning → LLM Call → Tool Execution |
|            ↳ Resource Access → Final Output               |
+-------------------------------------------------------------+
```

In [7]:
from langsmith import trace

prompt = 'Summarize the project scope and send an update to stakeholders via email and push notification.'

with trace("LangSmith Tracing Demo — Resources in Action - lang smith"):
    result = await Runner.run(agent, prompt)
    print(result.final_output)

The updates have been successfully sent:

- An email was dispatched to the stakeholders with the subject "DPoint AI Project Update."
- A push notification was also sent, announcing the exciting advancements in the DPoint AI platform.

If there's anything else you need or any other updates to make, just let me know!


# 🏁 Step 8 — Key Takeaways
✅ Tracing shows how agents reason and act step by step.
✅ Resources ground agents in real-world context.
✅ OpenAI Tracing = simple and native; LangSmith = advanced and production-grade.
✅ Combining both gives clarity and observability in Agentic AI systems.

🎓 Next: *Lab 6 – Integrate LangSmith Tracing and LangChain Agents!*